In [ ]:
!pip install -U langchain-ollama langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [ ]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [ ]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [ ]:
df.shape

(40000, 2)

In [ ]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")


In [ ]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'


In [ ]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [ ]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [ ]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.09 s, sys: 17.9 ms, total: 4.11 s
Wall time: 5.57 s


In [ ]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [ ]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 4.14 s, sys: 42.2 ms, total: 4.18 s
Wall time: 4.6 s


In [ ]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [ ]:
coffee_reviews_df.shape

(4981, 2)

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [ ]:
!ollama pull gemma3:270m

In [ ]:
!ollama list

NAME                                  ID              SIZE      MODIFIED       
gemma3:270m                           e7d36fb2c3b3    291 MB    1 second ago      
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    55 minutes ago    


In [ ]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [ ]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T13:07:35.043389706Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}},{"name":"scb10x/typhoon2.1-gemma3-4b:latest","model":"scb10x/typhoon2.1-gemma3-4b:latest","modified_at":"2025-10-19T12:11:37.329341704Z","size":2615662352,"digest":"8cfab9097c9d142fa16c7556826c9a7874f4642ac6d575e2d66a4f4979c48b76","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"3.9B","quantization_level":"Q4_K_M"}}]}

In [ ]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
llm = OllamaLLM(model="gemma3:270m")

In [ ]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีครับ/ค่ะ ยินดีให้บริการครับ/ค่ะ


In [ ]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [ ]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [ ]:
chain = prompt | llm

In [ ]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [ ]:
response

'```python\ndef find_sum_of_numbers(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    total = 0\n    for number in numbers:\n        total += number\n    return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum_of_numbers(numbers)\nprint(result)\n```\n\nโค้ดนี้มีตัวอย่างการใช้งานง่ายๆ ที่แสดงให้เห็นว่าสามารถเขียนโค้ดนี้เพื่อหาผลรวมของตัวเลขได้เช่นกัน\n'

In [ ]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(response))

```python
def find_sum_of_numbers(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    total = 0
    for number in numbers:
        total += number
    return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum_of_numbers(numbers)
print(result)
```

โค้ดนี้มีตัวอย่างการใช้งานง่ายๆ ที่แสดงให้เห็นว่าสามารถเขียนโค้ดนี้เพื่อหาผลรวมของตัวเลขได้เช่นกัน


In [ ]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [ ]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [ ]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [ ]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [ ]:
parser_chain = StrOutputParser()

In [ ]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [ ]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [ ]:
result

'วิธิีทำจากผลลัทธ์ต่อไปนี้ ถูกต้องแล้วค่ะ โจทย์ที่ถูกต้องคือ 2x = 10'

In [ ]:
import os
from langchain_openai import ChatOpenAI

In [ ]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [ ]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

result

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [ ]:
display(Markdown(formatted_result))

วิธิีทำจากผลลัทธ์ต่อไปนี้ ถูกต้องแล้วค่ะ โจทย์ที่ถูกต้องคือ 2x = 10

In [ ]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [ ]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [ ]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [ ]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 50.9 ms, sys: 3.95 ms, total: 54.8 ms
Wall time: 6.23 s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมไทย", "ขนมหวาน", "ฟรี", "ราคา", "ต้นทุน", "เบิ้ล", "ขนม", "ไทย", "ราคาสูง", "เลิกกิน", "กินแล้ว", "ขนมหวาน", "พร้อมกิน"]}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "มารยามัย",
    "ร้อน",
    "จิบ",
    "ฟัง",
    "พักผ่อน",
    "ดี",
    "อร่อย",
    "ราคาไม่แพง",
    "กินบ่อย",
    "จุกได้",
    "ซากอารยธรรมลาเต้",
    "เมพพพ",
    "ซีเค้ก",
    "แจ่ม",
    "ทาน",
    "อร่อย",
    "ทำทีละขั้นตอน"
  ]
}
```



In [ ]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [ ]:
!ollama list

NAME                                  ID              SIZE      MODIFIED           
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    2 seconds ago         
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago    


In [ ]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b"
)

In [ ]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 106 ms, sys: 11 ms, total: 117 ms
Wall time: 24.4 s


In [ ]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณระบุมาเพื่อดึงคำสำคัญจากรีวิวนี้:

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย

รีวิวนี้พูดถึงร้านอาหารขนาดใหญ่ที่มีจุดเริ่มต้นเป็นห้องน้ำ, อาหารบางอย่าง (ลาบไข่ต้ม, ทอดมันหัวปลี, พะแนงห่อไข่) และขนมหวานฟรี

ขั้นตอนที่ 2: ระบุหัวข้อหลักที่กล่าวถึง

*   ขนาดร้านอาหาร
*   ราคาอาหาร
*   รสชาติอาหาร (โดยเฉพาะทอดมันหัวปลีและพะแนงห่อไข่)
*   ขนมหวานฟรี

ขั้นตอนที่ 3: เลือก 3 keywords ที่สำคัญที่สุด

1.  ราคาแพง
2.  ทอดมันหัวปลี
3.  ขนมหวาน

ขั้นตอนที่ 4: ตอบในรูปแบบ JSON

```json
{
  "keywords": ["ราคาแพง", "ทอดมันหัวปลี", "ขนมหวาน"]
}
```

หวังว่าคำตอบนี้จะเป็นประโยชน์นะครับ หากคุณต้องการให้ผมช่วยอะไรเพิ่มเติม บอกได้เลยครับ

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text แน่นอนครับ ผมจะทำตามขั้นตอนที่คุณให้มาเพื่อดึงคำหลักจากรีวิวนี้ครับ

ขั้นตอนที่ 1: อ่านรีวิวและเข้าใจความหมาย


In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [ ]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 177 ms, sys: 8.74 ms, total: 186 ms
Wall time: 13.2 s


In [ ]:
results[0]['text']['keywords']

['ราคา', 'อาหาร', 'ขนาด']

In [ ]:
results[1]['text']['keywords']

['วาวี', 'กาแฟ', 'อารีย์']

In [ ]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 164 ms, sys: 13.4 ms, total: 177 ms
Wall time: 13.9 s


In [ ]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'อาหาร', 'รสชาติ'], ['กาแฟ', 'บรรยากาศ', 'รสชาติ'])

In [ ]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [ ]:
inputs = [{"review": r} for r in sample_reviews]

In [ ]:
inputs

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน '},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้าขอ Test ไม่สิ Taste เลยดีกว่าเดินเข้าร้านทันใด ป้าบบ...\n\n"พี่คับ เอา...ลาเต้ ร้อนแก้วนึงคับ"\nว่างๆ มานั่งจิบกาแฟ ฟังเพลง พักผ่อนได้ดีทีเดียว\n\nจ

In [ ]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 23.4 ms, sys: 1.23 ms, total: 24.6 ms
Wall time: 3.8 s


In [ ]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [ ]:
sample_reviews_100 = coffee_reviews_df['column0'].head(100).tolist()
inputs_100 = [{"review": r} for r in sample_reviews_100]

BATCH_SIZE = 8

results_100 = []

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0, len(inputs_100), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_100[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_100.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 13/13 [02:08<00:00,  9.85s/it]


In [ ]:
for result in results_100[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [ ]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [ ]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

In [ ]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [ ]:
context_extract = context_prompt | llm3 | context_parser

In [ ]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [ ]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [ ]:
results_100[0]['text']['keywords'].split(',')

['ราคา', ' บรรยากาศ', ' ไข่ต้ม']

In [ ]:
context_inputs = []

for review_data in tqdm(results_100, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 100/100 [00:00<00:00, 233405.90it/s]


In [ ]:
len(context_inputs)

300

In [ ]:
context_results = []
for i in tqdm(range(len(results_100)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_100[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 100/100 [09:40<00:00,  5.80s/it]


In [ ]:
results_100[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว',
   'บรรยากาศร้านใหญ่มากกกกกกก',
   'ไข่ต้ม']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี\nอย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ\nเขียวชอุ่ม\n\nว่าแล้วไม่รอช้

In [ ]:
sentiment_inputs = []

for review_data in tqdm(results_100, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 100/100 [00:00<00:00, 201843.31it/s]


In [ ]:
len(sentiment_inputs)

300

In [ ]:
sentiment_inputs[:6]

[{'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'บรรยากาศร้านใหญ่มากกกกกกก'},
 {'message': 'ไข่ต้ม'},
 {'message': 'กาแฟที่นี่อร่อยมากกกกกกก เรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง'},
 {'message': 'บรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสวนบ้านเราเอง ต้นไม้เยอะดี อย่างน้อยก็เป็นสีเขียวเล็กๆ ในใจกลางเมืองได้ละ เขียวชอุ่ม'},
 {'message': 'รสชาติและกลิ่นมันไม่ธรรมดาเลย หอมๆ ถือว่าเป็นมวยรุ่นเดียวกะ Starbuck ได้'}]

In [ ]:
for i in tqdm(range(len(results_100)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_100[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 100/100 [04:29<00:00,  2.70s/it]


In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_100
])

In [ ]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')

In [ ]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment

sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)
context_parser = JsonOutputParser(pydantic_object=ContexExtraction)
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [ ]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)


sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)

keyword_extract = keyword_prompt | llm3 | keywords_parser
context_extract = context_prompt | llm3 | context_parser
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [ ]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(1000).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [ ]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 125/125 [19:57<00:00,  9.58s/it]


In [ ]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords'].split(',')
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 1000/1000 [00:00<00:00, 200119.47it/s]


In [ ]:
context_results = []

for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list


Extracting contexts: 100%|██████████| 1000/1000 [1:13:11<00:00,  4.39s/it]


In [ ]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 1000/1000 [00:00<00:00, 238910.00it/s]


In [ ]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment:   0%|          | 0/1000 [00:00<?, ?it/s]


ConnectError: [Errno 111] Connection refused

In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])


In [ ]:
df.to_csv('results_1000.csv', index=False, encoding='utf-8-sig')